# MLP Classiifers DEF

  - 3. Syndigo Mapping MachineLearning

In [1]:
def unique_no_nan(x):
    return x.dropna().unique()

In [2]:
def constrcut_dic():
    key = unique_no_nan(syndigo_mapped['Level 1'])
    value = list( map(lambda x : re.sub('&|/', '', x), key))
    value = list( map(lambda x : re.sub(' +', '_', x), value))
    value = list(map(str.lower, value))
    #value = ['-'.join(['tfidf'+ str]), value]

    # print(f'key: {key}') 
    # print(f'value: {value}')
    return dict(zip(key, value)) 

In [3]:
def prepare_data_1():
    # Reading PIMMART data
    pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
        pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)
    
    # Reading Syndigo 259K data
    synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
        synd_ALL[i] = synd_ALL[i].astype(np.float64)
    
    # Stripping spaces from all columns
    df_obj = synd_ALL.select_dtypes(['object'])
    synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
    syndigo_mapped = synd_ALL
    pimmart = pim_gtin_mapped
    syndigo_mapped.drop_duplicates('GTIN', inplace = True)
    syndigo_mapped['ITEM_SUBCOM_text'] = \
    (syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()
    #syndigo_mapped['Level 1'].value_counts()
    print(f'syndigo_mapped.shape {syndigo_mapped.shape}') 
    #print(f'syndigo_mapped.info() {syndigo_mapped.info()}') 
    #print(f'syndigo_mapped.head() {syndigo_mapped.head()}') 
    #print(f'syndigo_mapped.columns {syndigo_mapped.columns}') 
    
    print(f"syndigo_mapped['Level 1'].value_counts() {syndigo_mapped['Level 1'].value_counts(dropna = False)}")    
    return syndigo_mapped, pim_gtin_mapped

In [4]:
def prepare_data_2(level_1):
    global   k_neighbors
    subset_df = syndigo_mapped[syndigo_mapped['Level 1'] == level_1]

    series =  subset_df['Level 2'].value_counts(ascending=True, dropna = False)
    k_neighbors = 5
    if min(series) < 10:
        k_neighbors = 3 
        drop_cut_off = 10
        subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series  < drop_cut_off ].index.tolist())].index) 
    print (f'Min cnt of Level 2 for class 1  {level_1}  is {min(series)} and k_neighbors is {k_neighbors}')
      
    x_subset_w2v = np.array(list(subset_df.ITEM_SUBCOM_text.apply(lambda x: get_item_vector(x.split(' ')))))                                            
    level2_id_map = dict(zip(subset_df['Level 2'].fillna('Other').unique(), range(subset_df['Level 2'].fillna('Other').nunique())))
    id2_level_map = dict(zip(range(subset_df['Level 2'].fillna('Other').nunique()), subset_df['Level 2'].fillna('Other').unique()))
    y_subset_w2v  = subset_df['Level 2'].fillna('Other').map(level2_id_map)
    
    return  x_subset_w2v, y_subset_w2v, subset_df, level2_id_map,  id2_level_map

In [5]:
#Build item vectors
def get_item_vector(item_vocab):
    vect = np.zeros_like(model.get_vector('chips'))
    for word in item_vocab:
        if word in model:
            vect += model.get_vector(word)
    return vect#/max(1,len(item_vocab))

In [40]:
def proc_pred_lr(level_1):
    lr_w2v = LogisticRegression(C = 100, multi_class = 'multinomial', solver = 'saga')
    #print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
    
    print(f"Training - w2v + LR \"{level_1}\"")
    warnings.filterwarnings("ignore", category=UserWarning)
    lr_w2v.fit(X_train, y_train)
    print(' Display LR test  metrics')
    preds_test = lr_w2v.predict(X_test)
    preds_train = lr_w2v.predict(X_train)
    print(classification_report(y_test, preds_test,labels = lr_w2v.classes_, target_names = [id2_level_map[i] for i in lr_w2v.classes_]))
    return preds_train, preds_test, lr_w2v

In [23]:
def proc_pred_mlp(level_1):

     mlp = MLPClassifier(hidden_layer_sizes=(200,), activation = 'relu', learning_rate= 'adaptive', early_stopping= True)
     #print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
     print(f"Training - w2v + MLP \"{level_1}\"")
     warnings.filterwarnings("ignore", category=UserWarning)
     mlp.fit(X_train, y_train)
     print(' Display MLP test  metrics')
     preds_test_mlp = mlp.predict(X_test)
     preds_train_mlp = mlp.predict(X_train)
    
     print(classification_report(y_test, preds_test_mlp,labels = mlp.classes_, target_names = [id2_level_map[i] for i in mlp.classes_]))      
     return preds_train_mlp, preds_test_mlp, mlp

In [7]:
def get_smp_strgy(dic_y_train, adj_ratio): 
    """
        get_smp_strgy  :  get sampling_strategy for SMOTE
    """
    smp_strgy = {} 
    
    adj_cnt =  int(max(dic_y_train.values()) * adj_ratio)
    for key, values in dic_y_train.items():
        if values < adj_cnt:
           smp_strgy[key] = adj_cnt
        else: 
            smp_strgy [key] = values
            
    return smp_strgy

In [51]:
def proc_smote(level_1): 
    print(f' smote level_1 : {level_1}')
    X_resampled, y_resampled = SMOTE(random_state= 42, sampling_strategy= smp_strgy, k_neighbors =   k_neighbors).fit_resample(X_train, y_train)
                                                                                             
    print('Before Resampling:\n',pd.Series(y_train).value_counts())
    print('After Resampling:\n',pd.Series(y_resampled).value_counts())
    mlp_smote = MLPClassifier(hidden_layer_sizes=(200,), activation = 'relu', learning_rate= 'adaptive', early_stopping= True)
    #print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
    print(f"Training - w2v + MLP \"{level_1}\"")
    warnings.filterwarnings("ignore", category=UserWarning)
    mlp_smote.fit(X_resampled, y_resampled)
    
    print(' Display MLP + SMOTE test  metrics')
    preds_test_mlp_smote = mlp_smote.predict(X_test)
    preds_train_mlp_smote = mlp_smote.predict(X_resampled)
    print(classification_report(y_test, preds_test_mlp_smote,labels = mlp_smote.classes_, target_names = [id2_level_map[i] for i in mlp_smote.classes_]))
    return preds_train_mlp_smote, preds_test_mlp_smote, mlp_smote

In [35]:
def validation(y_test, preds, level_1, model):
     global l_metrics      
     pgm = inspect.currentframe().f_code.co_name       
     print(f'{pgm} for model {model} with level_1 = {level_1}')
     accuracy = accuracy_score(y_test,preds)
     f1_score_micro    = f1_score(y_test, preds, average='micro')  
     f1_score_weighted = f1_score(y_test, preds, average="weighted")
     f1_score_macro    = f1_score(y_test, preds, average='macro')
     print (f"ccuracy Score = {accuracy},f1_score_micro = {f1_score_micro}, f1_score_macro = {f1_score_macro}, f1_score_weighted = {f1_score_weighted}")

     l_metric = [model, level_1, accuracy, f1_score_micro, f1_score_macro, f1_score_weighted ]
     l_metrics.append(l_metric)   
     return  f1_score_macro

In [43]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier,  RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import gensim
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import re, time, inspect, pickle
import logging

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
import re

# load  function of 'elapse_time'
path_code = 'C:\\users\\iny2819\\kroger\\Code\\'  
f_com_code = path_code + "com_code.py"
exec(compile(open(f_com_code , "rb").read(), f_com_code, 'exec' ))

In [10]:
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
DBFS = path
DBFO = path
DBFM = path
DBFR = path


In [11]:
syndigo_mapped, pim_gtin_mapped = prepare_data_1()

syndigo_mapped.shape (259085, 24)
syndigo_mapped['Level 1'].value_counts() Food / Beverages                            131593
Health & Beauty                              62259
Beer / Wine / Spirits                        17992
Cleaning & Janitorial                         9084
Livestock & Pet Supplies                      6325
Kitchen & Bathroom                            5268
Home & Venue Decoration                       3924
Toys / Games / Hobbies                        3191
Gardening & Outdoors                          2208
Childcare                                     2111
Office Supplies                               1671
Electronics                                   1499
Apparel                                       1283
Lighting & Fans                               1252
Tobacco Products                              1252
NaN                                           1132
Not classified                                1018
Hardware                                       923
Arts & 

In [12]:
dic_level1= constrcut_dic()


In [13]:
stop

NameError: name 'stop' is not defined

In [53]:
start_time = time.time() 

### import word2vec model trained on entire PIMMART data
#model = KeyedVectors.load_word2vec_format('/dbfs/FileStore/tables/DATA_SCIENCE/w2vmodel_053123_PIM_ALL.bin', binary=True)
model = KeyedVectors.load_word2vec_format(path + 'w2vmodel_053123_PIM_ALL.bin', binary=True)                                        
model.most_similar('chips')
adj_ratio = 0.2
i =0
err_smote_cnt = 0
err_smp_cnt = 0
#N = 5 
#dic_level1= constrcut_dic()
#dic_level1 = dict(list(dic_level1.items())[4: N])
l_metrics = []
for level_1, filenamee in dic_level1.items():    
    lr_f1_score_macro = 0
    mlp_f1_score_macro = 0
    mlp_smote_lr_f1_score_macro = 0
    print(f'{i}. Process level 1 = {level_1}')    
    i += 1    
    x_subset_w2v, y_subset_w2v, subset_df, level2_id_map,  id2_level_map =  prepare_data_2(level_1)
    if  len ( x_subset_w2v) < 10: 
        len_x_subset_w2v = len(x_subset_w2v)
        print(f'len(x_subset_w2v) = {len_x_subset_w2v}. Not  process len(x_subset_w2v) < 10')
        l_metrics.append(['N/A', level_1]) 
        continue
    try:
       X_train, X_test, y_train, y_test = train_test_split(x_subset_w2v, y_subset_w2v.values, test_size= 0.3, stratify=y_subset_w2v, random_state=42)
       dic_y_train = dict(pd.Series(y_train).value_counts())
        
       if len(dic_y_train.items())  == 1:
           print(f'Level1 {level_1} only has one level2. Will skip SMOTE' )
           l_metrics.append(['MLP_SMOTE N/A - only one Level 2', level_1]) 
       else: 
          preds_train_lr, preds_test_lr, lr_w2v = proc_pred_lr(level_1)
          lr_f1_score_macro = validation(y_test, preds_test_lr, level_1, model ='LR')
       preds_train_mlp, preds_test_mlp, mlp = proc_pred_mlp(level_1)
       mlp_f1_score_macro = validation(y_test, preds_test_mlp, level_1, model ='MLP')  
       
       
       if lr_f1_score_macro > mlp_f1_score_macro:
          winer = 'LR'
          model = lr_w2v
        else:
          winer = 'MLP'
          model = lr_w2v
       
       if len(dic_y_train.items())  == 1:
            print(f'Level1 {level_1} only has one level2. Will skip SMOTE' )
            l_metrics.append(['MLP_SMOTE N/A - only one Level 2', level_1]) 
            continue
       else:     
          smp_strgy = get_smp_strgy(dic_y_train, adj_ratio)
         
          try:
             preds_train_mlp_smote, preds_test_mlp_smote, mlp_smote =proc_smote(level_1)
             validation(y_test, preds_test_mlp_smote, level_1, model ='MLP_SMOTE')
          except Exception as smote_ve:
             err_smote_cnt += 1 
             logging.exception("number of smote error= %d.   Exception occurred for class_1 %s : %s", err_smote_cnt , level_1, str(smote_ve))
             continue
    except Exception as smp_ve:
       err_smp_cnt += 1 
       len_x_subset_w2v  = len ( x_subset_w2v)
       logging.exception("number of smaple error %d. len ( x_subset_w2v) = %d.  Exception occurred: %s", err_smp_cnt , len_x_subset_w2v,  str(smp_ve))
    
end_time = time.time()   
desc = f'Elapse_time for evluating LR, MLP and MLP_SMOTE'
elapse_time (  start_time, end_time, desc)

0. Process level 1 = Food / Beverages
Min cnt of Level 2 for class 1  Food / Beverages  is 309 and k_neighbors is 5
Training - w2v + LR "Food / Beverages"
 Display LR test  metrics
                                                          precision    recall  f1-score   support

                                    Dairy & Egg Products       0.89      0.88      0.88      2705
                                                 Grocery       0.91      0.96      0.93     22459
                                               Beverages       0.95      0.94      0.95      6372
Meat / Poultry / Seafood / Meat Substitutes (Perishable)       0.69      0.66      0.68      1193
                                            Frozen Foods       0.86      0.82      0.84      4074
                                           Bakery / Deli       0.56      0.29      0.39      1600
                                     Baby & Toddler Food       0.88      0.89      0.88       487
                                  

 Display MLP test  metrics
                                precision    recall  f1-score   support

                          Beer       0.95      0.94      0.95      1837
  Flavored Alcoholic Beverages       0.52      0.54      0.53       262
                          Wine       0.98      0.99      0.98      2134
                       Spirits       0.95      0.95      0.95      1067
               Ciders & Perrys       0.74      0.79      0.76        61
Alcoholic Beverage Mixed Packs       0.33      0.07      0.11        15
           Non-Alcoholic Beers       0.40      0.62      0.48        13
   Alcoholic Cordials & Syrups       0.00      0.00      0.00         4
       Alcohol Making Supplies       1.00      0.50      0.67         4

                      accuracy                           0.94      5397
                     macro avg       0.65      0.60      0.60      5397
                  weighted avg       0.94      0.94      0.94      5397

validation for model MLP with leve

 Display LR test  metrics
                                   precision    recall  f1-score   support

            Candles & Accessories       0.97      0.98      0.97       367
Special Occasion & Party Supplies       0.93      0.95      0.94       289
           Decorative Accessories       0.58      0.58      0.58        24
            Incense & Accessories       1.00      0.88      0.93         8
              Holiday Decorations       0.91      0.93      0.92       347
                           Linens       0.96      0.87      0.91       104
            Clocks & Thermometers       1.00      1.00      1.00         7
                  Wall Decoration       0.80      0.50      0.62         8
                Window Treatments       1.00      0.84      0.91        19

                         accuracy                           0.93      1173
                        macro avg       0.91      0.84      0.87      1173
                     weighted avg       0.93      0.93      0.93      11

Before Resampling:
 1    797
0    378
5    276
3    260
6    181
7    134
4    115
8     57
2     27
9      8
dtype: int64
After Resampling:
 1    797
0    378
5    276
3    260
6    181
7    159
8    159
9    159
4    159
2    159
dtype: int64
Training - w2v + MLP "Toys / Games / Hobbies"
 Display MLP + SMOTE test  metrics
                             precision    recall  f1-score   support

            Games & Puzzles       0.87      0.83      0.85       162
                Toy Figures       0.77      0.80      0.78       342
     Remote & Radio Control       0.38      0.45      0.42        11
          Baby Toys & Games       0.67      0.56      0.61       112
Role Playing & Pretend Play       0.37      0.37      0.37        49
    Construction & Modeling       0.81      0.78      0.80       118
               Toy Vehicles       0.74      0.71      0.73        77
       Outdoor Toys & Games       0.56      0.79      0.66        57
        Sports Toys & Games       0.50      0.48    

 Display MLP + SMOTE test  metrics
                      precision    recall  f1-score   support

            Clothing       0.96      0.96      0.96       259
      Safety Apparel       0.68      0.77      0.72        22
Personal Accessories       0.87      0.86      0.87        71
            Footwear       0.97      0.91      0.94        33

            accuracy                           0.92       385
           macro avg       0.87      0.87      0.87       385
        weighted avg       0.93      0.92      0.93       385

validation for model MLP_SMOTE with level_1 = Apparel
ccuracy Score = 0.9246753246753247. f1_score_micro = 0.9246753246753247. 
11. Process level 1 = Gardening & Outdoors
Min cnt of Level 2 for class 1  Gardening & Outdoors  is 133 and k_neighbors is 5
Training - w2v + LR "Gardening & Outdoors"
 Display LR test  metrics
                         precision    recall  f1-score   support

              Gardening       0.96      0.98      0.97       370
         Outd

Before Resampling:
 5    391
0    218
1    150
3    117
2     95
6     67
4     59
7     51
8     15
dtype: int64
After Resampling:
 5    391
0    218
1    150
3    117
2     95
6     78
7     78
4     78
8     78
dtype: int64
Training - w2v + MLP "Office Supplies"
 Display MLP + SMOTE test  metrics
                                    precision    recall  f1-score   support

   Greeting Cards & Gift Packaging       0.92      0.93      0.92        94
                        Stationery       0.75      0.78      0.77        65
                  Office Fasteners       0.79      0.75      0.77        40
                  Filing & Storage       0.80      0.78      0.79        50
     Stationery Cutters & Trimmers       0.63      0.48      0.55        25
 Writing Instruments & Accessories       0.91      0.92      0.91       168
Organizational Planning Stationery       0.63      0.68      0.66        28
            Shipping and Packaging       0.87      0.91      0.89        22
              

Before Resampling:
 5    291
3    181
0    171
4    115
2     79
1     76
8     53
7     32
6     25
9     10
dtype: int64
After Resampling:
 5    291
3    181
0    171
4    115
2     79
1     76
9     58
8     58
6     58
7     58
dtype: int64
Training - w2v + MLP "Electronics"
 Display MLP + SMOTE test  metrics
                               precision    recall  f1-score   support

                    Computers       0.78      0.84      0.81        74
        Video Games & Systems       0.67      0.81      0.73        32
      Cables & Cable Adapters       0.71      0.74      0.72        34
                Audio & Video       0.84      0.72      0.77        78
  Telephones & Communications       0.53      0.53      0.53        49
             Power Management       0.90      0.89      0.90       125
           Office Electronics       0.80      0.80      0.80        10
         Cameras & Camcorders       0.91      0.71      0.80        14
                 Data Storage       0.90     

 Display MLP + SMOTE test  metrics
                              precision    recall  f1-score   support

                   Fasteners       0.89      0.91      0.90       168
Adhesives & Sealing Products       0.90      0.81      0.85        47
                  Lubricants       0.75      1.00      0.86         3
             Keys & Padlocks       0.64      0.53      0.58        17
           Builders Hardware       0.67      0.70      0.68        20
             General Purpose       0.07      0.14      0.10         7
                Chain & Rope       0.67      0.25      0.36         8

                    accuracy                           0.81       270
                   macro avg       0.66      0.62      0.62       270
                weighted avg       0.83      0.81      0.82       270

validation for model MLP_SMOTE with level_1 = Hardware
ccuracy Score = 0.8148148148148148. f1_score_micro = 0.8148148148148148. 
22. Process level 1 = Books & Videos
Min cnt of Level 2 for cla

 Display MLP test  metrics
                                       precision    recall  f1-score   support

Paper / Plastic / Foam Plates & Bowls       0.96      0.88      0.92        81
          Paper / Plastic / Foam Cups       0.72      0.93      0.81        41
                     Plastic Flatware       0.80      0.60      0.69        20

                             accuracy                           0.85       142
                            macro avg       0.83      0.80      0.80       142
                         weighted avg       0.87      0.85      0.85       142

validation for model MLP with level_1 = Hospitality Supplies
ccuracy Score = 0.852112676056338. f1_score_micro = 0.852112676056338. 
 smote level_1 : Hospitality Supplies
Before Resampling:
 0    189
1     97
2     45
dtype: int64
After Resampling:
 0    189
1     97
2     45
dtype: int64
Training - w2v + MLP "Hospitality Supplies"
 Display MLP + SMOTE test  metrics
                                       precision

(0, 3, 7)

In [ ]:
err_cnt

## wip

In [ ]:
import logging
try:
       proc_smote()
except Exception as e:
       logging.error("Exception occurred", exc_info=True)
print("test")

In [ ]:
len(dic_level1 )

In [ ]:
syndigo_mapped.columns

In [ ]:
 k_neighbors

In [ ]:
subset_df['Level 2'].value_counts(ascending=True, dropna = False)

In [ ]:
syndigo_mapped[syndigo_mapped['Level 1']==level_1]['Level 2'].value_counts(ascending=True,  dropna = False)

In [ ]:

#syndigo_mapped['Level 1' ==  level_1]['Level 2'].value_counts(dropna = False) 
min_cnt = min(syndigo_mapped[syndigo_mapped['Level 1']==level_1]['Level 2'].value_counts(ascending=True))
k_neighbors = 5
if min_cnt < 3 :  k_neighbors  +  min_cnt  

In [ ]:
level_1 = 'Health & Beauty'.

In [ ]:
syndigo_mapped['Level 1' ==  level_1]['Level 2'].value_counts() 

In [ ]:
min(subset_df['Level 2'].value_counts(ascending=True))

In [ ]:
series =  subset_df['Level 2'].value_counts(ascending=True, dropna = False)

In [ ]:

if min(series) < 3:
    k_neighbors = 3
    subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series  < k_neighbors].index.tolist())].index) 

In [ ]:
#type(X_train) # numpy.ndarray
len(X_train) #28

In [ ]:


dic_level1= constrcut_dic()

dic_level1 = dict(list(dic_level1.items())[4: 5])

for level_1, filenamee in dic_level1.items():
    i += 1
    print(f'{i}. Process level 1 = {level_1}')
    x_subset_w2v, y_subset_w2v, subset_df, level2_id_map,  id2_level_map =  prepare_data_2(level_1)
    X_train, X_test, y_train, y_test = train_test_split(x_subset_w2v, y_subset_w2v.values, test_size= 0.2, stratify=y_subset_w2v, random_state=42)
    proc_predict(level_1)
    dic_y_train = dict(pd.Series(y_train).value_counts())
    smp_strgy = get_smp_strgy(dic_y_train, adj_ratio)
    proc_smote()


In [ ]:
unique, couints = np.unique(y_train, return_counts = True)
y_dic =dict(zip(unique, couints )) 
y_dic

In [14]:
def proc_smotex(level_1): 
    err_smote_cnt = 0
    err_fit_cnt = 0
    print(f' smotex level_1 : {level_1}')
    try:
         X_resampled, y_resampled = SMOTE(random_state= 42, sampling_strategy= smp_strgy, k_neighbors =   k_neighbors).fit_resample(X_train, y_train)
                                                                                                  
         print('Before Resampling:\n',pd.Series(y_train).value_counts())
         print('After Resampling:\n',pd.Series(y_resampled).value_counts())
         mlp_smote = MLPClassifier(hidden_layer_sizes=(200,), activation = 'relu', learning_rate= 'adaptive', early_stopping= True)
         #print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
         print(f"Training - w2v + MLP + SMOTE for \"{level_1}\"")
         warnings.filterwarnings("ignore", category=UserWarning)
         try:   
            mlp_smote.fit(X_resampled, y_resampled)
            
            print(' Display MLP + SMOTE test  metrics')
            preds_test_mlp_smote = mlp_smote.predict(X_test)
            preds_train_mlp_smote = mlp_smote.predict(X_resampled)
            print(classification_report(y_test, preds_test_mlp_smote,labels = mlp_smote.classes_, target_names = [id2_level_map[i] for i in mlp_smote.classes_]))
         except Exception as fit_ve:
              err_fit_cnt += 1  
              loggifitng.exception("proc_smotex: number of fit error= %d.   Exception occurred for class_1 %s : %s", err_fit_cnt , level_1, str(fite_ve))
              return
    except Exception as smote_ve:
          err_smote_cnt += 1 
          logging.exception("proc_smotex: number of smote error= %d.   Exception occurred for class_1 %s : %s", err_smote_cnt , level_1, str(smote_ve))
        
    return

In [ ]:
def wrapper(dic_srt, dic_end):
    ### import word2vec model trained on entire PIMMART data
    from gensim.models import Word2Vec, KeyedVectors
    #model = KeyedVectors.load_word2vec_format('/dbfs/FileStore/tables/DATA_SCIENCE/w2vmodel_053123_PIM_ALL.bin', binary=True)
    model = KeyedVectors.load_word2vec_format(path + 'w2vmodel_053123_PIM_ALL.bin', binary=True)                                        
    model.most_similar('chips')
    adj_ratio = 0.2
    i =0
    err_smp_cnt = 0
    err_smote_cnt = 0
    N = 5 
    dic_level1= constrcut_dic()
    dic_level1 = dict(list(dic_level1.items())[int(dic_srt): int(dic_end)])
    
    for level_1, filenamee in dic_level1.items():
        i += 1
        print(f'{i}. Process level 1 = {level_1}')
        x_subset_w2v, y_subset_w2v, subset_df, level2_id_map,  id2_level_map =  prepare_data_2(level_1)
        if  len ( x_subset_w2v) < 10: 
            len_x_subset_w2v = len(x_subset_w2v)
            print(f'len(x_subset_w2v) = {len_x_subset_w2v}. Not  process len(x_subset_w2v) < 10')
            continue
        try:
           X_train, X_test, y_train, y_test = train_test_split(x_subset_w2v, y_subset_w2v.values, test_size= 0.3, stratify=y_subset_w2v, random_state=42)
    
           proc_predict(level_1)
           dic_y_train = dict(pd.Series(y_train).value_counts())
           smp_strgy = get_smp_strgy(dic_y_train, adj_ratio)
          
           try:
              proc_smotex(level_1)
           except Exception as smote_ve:
              err_smote_cnt += 1 
              logging.exception("number of smote error= %d.   Exception occurred for class_1 %s : %s", err_smote_cnt , level_1, str(smote_ve))
              continue
        except Exception as smp_ve:
           err_smp_cnt += 1 
           len_x_subset_w2v  = len ( x_subset_w2v)
           logging.exception("number of smaple error %d. len ( x_subset_w2v) = %d.  Exception occurred: %s", err_smp_cnt , len_x_subset_w2v,  str(smp_ve))
        return  dic_level1, level_1

In [ ]:
dic_level1= constrcut_dic()
dic_level1.items()

In [ ]:
 dic_level1, level1 = wrapper(dic_srt= 13, dic_end= 14)

In [41]:
validation(y_test, preds_train_lr, level_1, model ='LR')

validation for model LR with level_1 = Food / Beverages
ccuracy Score = 0.8922944424742895. f1_score_micro = 0.8922944424742895. 


In [37]:
len(y_test)

39478

In [45]:
validation(y_test, preds_test_mlp_smote, level_1, model ='MLP_SMOTE')

NameError: name 'preds_test_mlp_smote' is not defined

In [54]:
l_metrics

[['LR', 'Food / Beverages', 0.8923704341658645, 0.8923704341658645],
 ['MLP', 'Food / Beverages', 0.907898069811034, 0.907898069811034],
 ['MLP_SMOTE', 'Food / Beverages', 0.8935863012310654, 0.8935863012310654],
 ['LR', 'Health & Beauty', 0.9165863582824714, 0.9165863582824714],
 ['MLP', 'Health & Beauty', 0.9258485919263304, 0.9258485919263304],
 ['MLP_SMOTE', 'Health & Beauty', 0.924777813470393, 0.9247778134703931],
 ['LR', 'Beer / Wine / Spirits', 0.9375579025384473, 0.9375579025384473],
 ['MLP', 'Beer / Wine / Spirits', 0.9366314619232907, 0.9366314619232907],
 ['MLP_SMOTE',
  'Beer / Wine / Spirits',
  0.9355197331851028,
  0.9355197331851028],
 ['LR', 'Kitchen & Bathroom', 0.9867172675521821, 0.9867172675521821],
 ['MLP', 'Kitchen & Bathroom', 0.9867172675521821, 0.9867172675521821],
 ['MLP_SMOTE', 'Kitchen & Bathroom', 0.9854522454142948, 0.9854522454142948],
 ['LR', 'Cleaning & Janitorial', 0.9496508636530687, 0.9496508636530687],
 ['MLP', 'Cleaning & Janitorial', 0.948548327

In [49]:
    lr_w2v = LogisticRegression(C = 100, multi_class = 'multinomial', solver = 'lbfgs')
    #print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
    
    print(f"Training - w2v + LR \"{level_1}\"")
    warnings.filterwarnings("ignore", category=UserWarning)
    lr_w2v.fit(X_train, y_train)

Training - w2v + LR "Plumbing & Water Service"


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

In [50]:
 preds_train_mlp_smote, preds_test_mlp_smote =proc_smote(level_1)

 smote level_1 : Plumbing & Water Service


ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead